#### base랑 engine 은 main함수같은 역할..! 맨앞에 항상 선언해주기.

In [26]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
engine = create_engine("sqlite:///", echo=True)
Base = declarative_base()

In [27]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship
Session = sessionmaker(bind = engine)
session = Session()

In [28]:
class Artist(Base):
    __tablename__ = "artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    album = relationship("Album", back_populates="artist")
    # album 속성은 Album 클래스의 artist속성과 관계가 있다.
    # Artist는 여러개의 album을 가질 수 있음.
    
    def __repr__(self):
        return "<T'artist(name='%s')>" %(self.name)
    
    
class Album(Base):
    __tablename__ = "album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    
    artist_id = Column(Integer, ForeignKey("artist.id"))
    artist = relationship("Artist", back_populates = "album", uselist=False)
    # artist라는 속성은 Artist 클래스의 album속성과 관계가 있다.
    # N:M관계가 기본이지만, Ablum이 하나의 Artist만 가져야 하므로 1:N의 관계에서 1이 되어야 하는 부분에  uselist=False 넣어줘야함.
    # default가 True. M;N임.

    
    
class Genre(Base):
    __tablename__ = "genre"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    extend_existing=True
    
    
    
class Track(Base):
    __tablename__ = "track"
    id = Column(Integer, primary_key=True)
    title = Column(String)
    extend_existing=True
    album_id = Column(Integer, ForeignKey("album.id"))
    genre_id = Column(Integer, ForeignKey("genre.id"))

In [29]:
Base.metadata.create_all(engine)

2018-07-12 11:16:45,896 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-12 11:16:45,896 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 11:16:45,896 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-12 11:16:45,896 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 11:16:45,896 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-12 11:16:45,896 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 11:16:45,896 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-12 11:16:45,911 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 11:16:45,911 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-12 11:16:45,911 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 11:16:45,911 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-12 11:16:45,911 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 11:16:45,911 INFO sqlalchemy.engin

In [30]:
artist1 = Artist(name="Led Zeplin")
artist2 = Artist(name="AC/DC")

# Relationship설정.
artist1.album = [Album(title="IV"), Album(title = "Whoe Made Who")]
artist1.album[0].genre.track = 
# 인스턴스에다가 값 넣듯이 그냥 넣을 수 있게됨..!
# db관계 생각할필요없고, class만생성해달라고하면됨.
# PK와 FK가 누군지만 알고,
# 값넣고, commit만해주면됨. 정말편한거에용 이거!
# class사이의 관계를 relationship가지고 표현만해주면됨.
# 나머지는 metadata, engine, base라는 애가 코어단에서 알아서 해줄거야.

In [31]:
session.add(artist1)
session.commit()

2018-07-12 11:16:49,463 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 11:16:49,463 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-12 11:16:49,463 INFO sqlalchemy.engine.base.Engine ('Led Zeplin',)
2018-07-12 11:16:49,478 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-12 11:16:49,478 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2018-07-12 11:16:49,478 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-12 11:16:49,478 INFO sqlalchemy.engine.base.Engine ('Whoe Made Who', 1)
2018-07-12 11:16:49,478 INFO sqlalchemy.engine.base.Engine COMMIT


In [32]:
print(artist1.id, artist1.name, artist1.album[0].id)

#기본적으로 N:M관계니까 artist.album.title는 list타입임.

2018-07-12 11:16:51,399 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 11:16:51,399 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-12 11:16:51,399 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-12 11:16:51,399 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-07-12 11:16:51,399 INFO sqlalchemy.engine.base.Engine (1,)
1 Led Zeplin 1


In [33]:
# 값 바꾸기.

artist1.album[0].title = "test"

#class를 가지고 db랑 상관없이 메모리상에서 값 변경이 이뤄지고있는중
# db랑 싱크 맞추려면 commit한버해주고.

In [ ]:
session.dirty
session.new

In [35]:
artist1.name="xyz"